In [2]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [3]:
installs = pd.read_csv('data/installs.csv.gzip', low_memory=False, compression='gzip', parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
created,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,8409380583076227678,NaN
2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,806493034070542041,NaN
2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,NaN,False,True,6287817205707153877,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,False,NaN,6319394158544413414,6.977049e+18
2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,4206683315615624585,5.221863e+18
2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,NaN,True,NaN,8335834010411202437,7.528974e+18


In [25]:
label1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'})

In [26]:
label1['created'] = label1['created'] - pd.Timestamp('2019-04-21 00:00:00')
label1['created'] = label1['created'].transform(lambda x: x.total_seconds())

In [27]:
label1 = label1.reset_index()
label1 = label1.merge(installs['2019-04-18':'2019-04-20'].reset_index(),on="ref_hash", how="inner")
label1 = label1.rename(columns={'created_x':'label','created_y':'created'})
label1

,ref_hash,label,created,application_id,ref_type,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2493454867357963,19452.720,2019-04-20 22:22:49.792,158,1891515180541284343,NaN,False,False,6287817205707153877,NaN,1.327864e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,8426419977387901158,3.301378e+18
1,3589607969461633,108671.325,2019-04-18 14:57:36.658,77,1891515180541284343,NaN,False,False,6287817205707153877,NaN,7.060110e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,1036806782641860771,3.301378e+18
2,3589607969461633,108671.325,2019-04-18 16:16:56.283,302,1891515180541284343,NaN,False,False,6287817205707153877,3.083059e+17,7.060110e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.1.0; SM-J610...,NaN,NaN,False,NaN,1036806782641860771,6.977049e+18
3,3861795021724487,252890.111,2019-04-20 18:13:48.197,339,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,8499290145871692375,8.441417e+18
4,6455321535851721,217290.613,2019-04-19 03:30:32.429,94,1891515180541284343,NaN,False,False,6287817205707153877,NaN,6.760982e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4458082048655699359,3.301378e+18
5,7944594974253876,220770.040,2019-04-19 17:50:36.108,327,1891515180541284343,NaN,False,False,6287817205707153877,NaN,NaN,Dalvik/2.1.0 (Linux; U; Android 9; SM-N960U1 B...,NaN,NaN,NaN,NaN,NaN,1657122989587830217,NaN
6,14664043499550181,97507.843,2019-04-18 14:53:21.834,94,1891515180541284343,NaN,False,False,6287817205707153877,NaN,2.827096e+17,adjust.com,NaN,NaN,NaN,NaN,NaN,2676767428799963998,3.301378e+18
7,15057990565785689,238575.817,2019-04-19 18:59:14.898,121,1891515180541284343,NaN,False,True,6287817205707153877,4.567867e+18,1.292000e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; BLADE V...,1eb2938a-29f0-417c-a433-7a39e21f94f3,Open,False,NaN,2936634590955379497,6.977049e+18
8,15057990565785689,238575.817,2019-04-19 18:59:16.926,121,1891515180541284343,NaN,False,False,6287817205707153877,4.567867e+18,1.292000e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; BLADE V...,NaN,NaN,False,NaN,2936634590955379497,6.977049e+18
9,15695176139879422,239972.035,2019-04-18 06:49:51.272,94,1891515180541284343,NaN,False,False,6287817205707153877,NaN,3.017401e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,1637090004618393412,3.301378e+18


In [28]:
y=label1.loc[:,'label']
y


0        19452.720
1       108671.325
2       108671.325
3       252890.111
4       217290.613
5       220770.040
6        97507.843
7       238575.817
8       238575.817
9       239972.035
10      170757.389
11      243918.623
12       22624.206
13      225764.103
14      225764.103
15      107517.631
16      152442.672
17      152442.672
18       93014.879
19      245073.483
20      166275.213
21       11104.805
22       11104.805
23       11104.805
24        3211.751
25       48230.003
26       48230.003
27       48230.003
28      189169.026
29      103646.069
           ...    
4917    152749.422
4918    188158.560
4919    188158.560
4920    224769.979
4921     56883.456
4922    153368.004
4923    185301.761
4924    150977.707
4925    225539.100
4926     61272.875
4927    185807.476
4928      5038.878
4929     97523.833
4930    226109.082
4931    175652.554
4932    175652.554
4933    246426.227
4934    246426.227
4935     84973.446
4936    147634.249
4937    147634.249
4938    1864

In [29]:
data1=label1.drop('label',axis=1)
data1.head()
#Aca se debe hacer limpieza de los datos... dejar features que sirven y hacer nuevos

,ref_hash,created,application_id,ref_type,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2493454867357963,2019-04-20 22:22:49.792,158,1891515180541284343,NaN,False,False,6287817205707153877,NaN,1.327864e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,8426419977387901158,3.301378e+18
1,3589607969461633,2019-04-18 14:57:36.658,77,1891515180541284343,NaN,False,False,6287817205707153877,NaN,7.060110e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,1036806782641860771,3.301378e+18
2,3589607969461633,2019-04-18 16:16:56.283,302,1891515180541284343,NaN,False,False,6287817205707153877,3.083059e+17,7.060110e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.1.0; SM-J610...,NaN,NaN,False,NaN,1036806782641860771,6.977049e+18
3,3861795021724487,2019-04-20 18:13:48.197,339,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,8499290145871692375,8.441417e+18
4,6455321535851721,2019-04-19 03:30:32.429,94,1891515180541284343,NaN,False,False,6287817205707153877,NaN,6.760982e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4458082048655699359,3.301378e+18


In [30]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4947 entries, 0 to 4946
Data columns (total 18 columns):
ref_hash              4947 non-null int64
created               4947 non-null datetime64[ns]
application_id        4947 non-null int64
ref_type              4947 non-null int64
click_hash            18 non-null object
attributed            4947 non-null bool
implicit              4947 non-null bool
device_countrycode    4947 non-null int64
device_brand          2814 non-null float64
device_model          4711 non-null float64
session_user_agent    4820 non-null object
user_agent            3174 non-null object
event_uuid            1160 non-null object
kind                  1160 non-null object
wifi                  2923 non-null object
trans_id              93 non-null object
ip_address            4947 non-null int64
device_language       4716 non-null float64
dtypes: bool(2), datetime64[ns](1), float64(3), int64(5), object(7)
memory usage: 826.7+ KB


In [1]:
#Vamos a dejar las columnas que nos aporten informacion...

In [41]:
# Podemos "inventar" que los que no tienen info son falsos...
data1['wifi']=data1['wifi'].fillna(False)

In [39]:
data1.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','trans_id','event_uuid'],axis =1, inplace = True )


In [42]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4947 entries, 0 to 4946
Data columns (total 10 columns):
ref_hash              4947 non-null int64
application_id        4947 non-null int64
attributed            4947 non-null bool
implicit              4947 non-null bool
device_countrycode    4947 non-null int64
device_brand          2814 non-null float64
device_model          4711 non-null float64
wifi                  4947 non-null bool
ip_address            4947 non-null int64
device_language       4716 non-null float64
dtypes: bool(3), float64(3), int64(4)
memory usage: 483.7 KB


In [44]:
y.head(1)

0    19452.72
Name: label, dtype: float64

In [45]:
data1=data1.fillna(0)

In [46]:
data1.shape,y.shape

((4947, 10), (4947,))

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(data1, y, test_size=0.2, random_state=123)

In [48]:
reg=linear_model.LinearRegression()

In [49]:
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [65]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3957, 10), (990, 10), (3957,), (990,))

In [50]:
pred=reg.predict(X_test)

In [34]:
pred[3]

119232.0

In [64]:
pred.shape

(990,)

In [32]:
y_test

3063    243153.456
918     185233.034
655      21086.786
1502     73309.262
934     183573.777
730      11865.516
31      231064.121
3242     92701.601
3386    107372.837
1084    258992.418
1451     96195.132
4927    185807.476
4289      8037.325
4382    200965.498
239     167946.399
1589     21355.258
1945     59522.470
3091    142783.277
4892      6400.576
761      17125.551
748     116016.453
3838      3244.328
3118    175395.043
439      20047.465
2322     66040.100
4559    102165.108
4115     51349.643
3793    205955.717
2022     54555.023
4798    136842.335
           ...    
3855    156376.743
682      14996.289
4284      9722.838
3927    250419.219
2493    223655.174
4812    158080.085
1914     32244.325
2997     19593.091
1824     62256.247
1179     23825.569
1064    105945.850
3649     87210.832
827     190862.372
4928      5038.878
1370    182485.388
2328     78377.696
1459    258542.142
585     219239.485
2676    133308.203
3341     25720.994
2070      6255.778
1033    1759

In [35]:
# mean square error
np.mean((pred-y_test)**2)

5617448330.451362

#Ahora estas predicciones las agregamos como feature para la segunda ventana...

Empezamos con los nuevos labels


In [55]:
label2 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})

In [56]:
label2['created'] = label2['created'] - pd.Timestamp('2019-04-24 00:00:00')
label2['created'] = label2['created'].transform(lambda x: x.total_seconds())

In [57]:
label2 = label2.reset_index()
label2 = label2.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="inner")
label2 = label2.rename(columns={'created_x':'label','created_y':'created'})
label2

,ref_hash,label,created,application_id,ref_type,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,622102439689666,979.059,2019-04-23 23:17:02.399,121,1891515180541284343,NaN,False,True,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,f8b3f19d-c761-4d6a-9f20-4e1832d38146,Open,True,NaN,1848802948798920304,6.977049e+18
1,622102439689666,979.059,2019-04-23 23:17:02.787,121,1891515180541284343,NaN,False,False,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,NaN,NaN,True,NaN,1848802948798920304,6.977049e+18
2,928188758790177,247212.597,2019-04-21 01:50:16.400,68,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Grability/17147 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,3483843913495440783,8.441417e+18
3,4129436938177104,229521.524,2019-04-21 20:56:18.807,117,1891515180541284343,NaN,False,False,6287817205707153877,NaN,2.399584e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,5840064274212175308,3.301378e+18
4,5133475695351500,209187.715,2019-04-22 23:38:06.589,194,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Gardenscapes/3.3.2 CFNetwork/978.0.7 Darwin/18...,NaN,NaN,True,NaN,2437566759560000641,8.441417e+18
5,7881267125156197,176402.652,2019-04-22 14:15:28.970,210,1891515180541284343,NaN,False,False,6287817205707153877,NaN,8.479522e+18,http-kit/2.0,Mozilla/5.0 (Linux; Android 9; SM-G970F Build/...,NaN,NaN,NaN,NaN,97224138037576704,3.301378e+18
6,7944594974253876,9055.796,2019-04-23 13:19:30.040,281,1891515180541284343,NaN,False,False,6287817205707153877,NaN,NaN,Dalvik/2.1.0 (Linux; U; Android 9; SM-N960U1 B...,NaN,NaN,NaN,NaN,NaN,4063933476320529056,NaN
7,13912050945871915,52090.338,2019-04-23 12:54:17.555,226,1891515180541284343,NaN,False,False,6287817205707153877,4.567867e+18,1.292000e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; BLADE V...,NaN,NaN,False,NaN,3645808844944603776,6.977049e+18
8,24294411704078426,182902.601,2019-04-23 03:02:23.309,36,1891515180541284343,NaN,False,False,6287817205707153877,3.083059e+17,9.186120e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; SM-G532...,NaN,NaN,True,NaN,4950519528220185340,6.977049e+18
9,24625906561621678,225275.348,2019-04-22 15:08:28.648,226,1891515180541284343,NaN,False,False,6287817205707153877,2.208835e+18,4.073186e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.1.0; moto e5...,NaN,NaN,False,NaN,653240972862608991,6.977049e+18


In [58]:
y2=label2.loc[:,'label']


0          979.059
1          979.059
2       247212.597
3       229521.524
4       209187.715
5       176402.652
6         9055.796
7        52090.338
8       182902.601
9       225275.348
10      225275.348
11       60963.993
12      237204.118
13      237204.118
14        7428.966
15      162425.285
16      151460.553
17      109368.574
18       69484.391
19       60461.551
20       74326.998
21      131427.750
22       53445.484
23      183558.105
24      251205.235
25       59329.302
26      255402.101
27      229798.099
28      241053.665
29      241053.665
           ...    
4847    196892.915
4848    196892.915
4849    101066.591
4850    180470.367
4851    220695.034
4852    241297.212
4853    241297.212
4854     12533.996
4855     12533.996
4856    113130.331
4857    198149.001
4858    183837.613
4859    235707.018
4860    241096.675
4861    241096.675
4862    180657.109
4863    180657.109
4864     90178.277
4865    192165.869
4866     13016.591
4867    223988.523
4868    1656

In [59]:
data2=label2.drop('label',axis=1)
data2.head()

,ref_hash,created,application_id,ref_type,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,622102439689666,2019-04-23 23:17:02.399,121,1891515180541284343,NaN,False,True,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,f8b3f19d-c761-4d6a-9f20-4e1832d38146,Open,True,NaN,1848802948798920304,6.977049e+18
1,622102439689666,2019-04-23 23:17:02.787,121,1891515180541284343,NaN,False,False,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,NaN,NaN,True,NaN,1848802948798920304,6.977049e+18
2,928188758790177,2019-04-21 01:50:16.400,68,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Grability/17147 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,3483843913495440783,8.441417e+18
3,4129436938177104,2019-04-21 20:56:18.807,117,1891515180541284343,NaN,False,False,6287817205707153877,NaN,2.399584e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,5840064274212175308,3.301378e+18
4,5133475695351500,2019-04-22 23:38:06.589,194,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Gardenscapes/3.3.2 CFNetwork/978.0.7 Darwin/18...,NaN,NaN,True,NaN,2437566759560000641,8.441417e+18


In [63]:
data2.shape,pred.shape

((4877, 18), (990,))

In [68]:
pred

array([117760., 121984., 120064., 119296., 116864., 116224., 123520.,
       117632., 116480., 121216., 118016., 112128., 116096., 112768.,
       117376., 121472., 121216., 116736., 113408., 115200., 121600.,
       117120., 117504., 119808., 118400., 114048., 117760., 113024.,
       123008., 115968., 117504., 114944., 121472., 120960., 117248.,
       117376., 119680., 112384., 118016., 112768., 117632., 112768.,
       114816., 117248., 118784., 115840., 119168., 118784., 111744.,
       120576., 117248., 115968., 118784., 116608., 114048., 118016.,
       124288., 115840., 116864., 119424., 120704., 121728., 115584.,
       119168., 117632., 116480., 116736., 114816., 113408., 111360.,
       118016., 118016., 116352., 121984., 117376., 118272., 120832.,
       120960., 114944., 117376., 124544., 118528., 113664., 120320.,
       120448., 122112., 121728., 118016., 114432., 114432., 116992.,
       117120., 123904., 123904., 115456., 116736., 118528., 121856.,
       117632., 1140

In [74]:
#Esas pred se las tenemos que agregar al set de entrenamiento ...
#¿Como hacemos? solo tenemos 990 filas! rellenamos con algun numero? o sea 999999?
#Tambien podemos usar el label de la anterior... para esta
data2['label1']=y
data2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4877 entries, 0 to 4876
Data columns (total 19 columns):
ref_hash              4877 non-null int64
created               4877 non-null datetime64[ns]
application_id        4877 non-null int64
ref_type              4877 non-null int64
click_hash            13 non-null object
attributed            4877 non-null bool
implicit              4877 non-null bool
device_countrycode    4877 non-null int64
device_brand          2709 non-null float64
device_model          4571 non-null float64
session_user_agent    4755 non-null object
user_agent            3262 non-null object
event_uuid            1252 non-null object
kind                  1252 non-null object
wifi                  2919 non-null object
trans_id              89 non-null object
ip_address            4877 non-null int64
device_language       4571 non-null float64
label1                4877 non-null float64
dtypes: bool(2), datetime64[ns](1), float64(4), int64(5), object(7)
memory us

In [76]:
data2.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','trans_id','event_uuid'],axis =1, inplace = True )
data2.fillna(0)


ValueError: labels ['created' 'ref_type' 'click_hash' 'session_user_agent' 'user_agent'
 'kind' 'trans_id' 'event_uuid'] not contained in axis

In [77]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4877 entries, 0 to 4876
Data columns (total 11 columns):
ref_hash              4877 non-null int64
application_id        4877 non-null int64
attributed            4877 non-null bool
implicit              4877 non-null bool
device_countrycode    4877 non-null int64
device_brand          2709 non-null float64
device_model          4571 non-null float64
wifi                  2919 non-null object
ip_address            4877 non-null int64
device_language       4571 non-null float64
label1                4877 non-null float64
dtypes: bool(2), float64(4), int64(4), object(1)
memory usage: 550.5+ KB


In [78]:
data2.shape,y2.shape

((4877, 11), (4877,))

In [79]:
X2_train, X2_test, y2_train, y2_test = \
    train_test_split(data2, y2, test_size=0.2, random_state=123)

In [81]:
reg2=linear_model.LinearRegression()


In [88]:
X2_train=X2_train.fillna(0)
y2_train=y2_train.fillna(0)
X2_test=X2_test.fillna(0)


In [86]:
reg2.fit(X2_train,y2_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [89]:
X2_train.shape,X2_test.shape,y2_train.shape,y2_test.shape
pred2=reg2.predict(X2_test)
np.mean((pred2-y2_test)**2)


5717451376.098835